# Torch code
Used to learn the weights and biases

## Training the weights and biases

In [1]:
# CNN quantized to INT8
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

torch.manual_seed(42)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(1152, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()


    def forward(self, x):
        x = self.quant(x)
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.dequant(x)
        x = self.softmax(x)

        return x

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
model.qconfig = torch.ao.quantization.default_qconfig
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model_quantized = torch.ao.quantization.prepare_qat(model,inplace=False)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

model_quantized = torch.ao.quantization.convert(model_quantized)
model_quantized

# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 485kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.42MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.54MB/s]


Epoch 1, Loss: 1.5594
Epoch 2, Loss: 1.4836
Epoch 3, Loss: 1.4800
Epoch 4, Loss: 1.4768
Epoch 5, Loss: 1.4749


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/utils.py:408: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


Test Accuracy: 98.60%


In [2]:
fcl_weights=[torch.int_repr(model_quantized.fc1.weight().cpu()).numpy(),torch.int_repr(model_quantized.fc2.weight().cpu()).numpy()]
fcl_weights

[array([[ -95,   28,   -5, ...,  -94,   62, -117],
        [  11, -104,  -63, ...,   87,   15,  114],
        [ -30,  118,  126, ...,  -89,   -9,  -39],
        ...,
        [ -82,   11,   36, ...,  -93,  -26,  -69],
        [  54,   74,   46, ..., -123,   82,  125],
        [  71,  126,   -1, ...,   27,  -60,  -52]], dtype=int8),
 array([[-120,   -5,  -15, ...,   36,   23, -124],
        [ -37,   23,  -15, ...,  -87,  -27,   15],
        [ -58,   33,   42, ...,   24,   36, -124],
        ...,
        [ 106,  -77,   58, ..., -104,  -96,  -38],
        [-123,  -49, -122, ...,   72,  -85,   34],
        [ -82,  -85,   16, ...,   62,  -71,  105]], dtype=int8)]

In [3]:
import numpy as np
biases=[model_quantized.fc1.bias().detach().cpu().numpy(),model_quantized.fc2.bias().detach().cpu().numpy()]
fcl_biases=[np.int8(biases[0]*100),np.int8(biases[1]*100)]
fcl_biases

[array([ 2,  0,  0,  0,  0,  0,  2, -1,  1,  1,  0,  2,  2,  1, -2, -1, -1,
        -1,  0,  0,  1,  0,  2,  2,  0, -1, -2,  0, -1, -2, -2,  0,  0,  0,
         0,  1, -2,  1, -2, -2,  1,  0,  1, -2,  0, -1,  1,  1,  0, -1,  1,
         0,  2,  2, -2, -1,  2,  0,  0,  2,  2,  1,  2,  0,  1,  1,  0, -1,
         1, -2,  0, -2,  2, -2, -1,  1,  2,  2,  2,  0,  0,  0,  1,  2, -1,
         0, -2,  1,  0,  0,  1,  2,  2, -1,  0,  2, -2,  1,  2,  1, -1, -1,
         2,  1,  1, -2,  2, -2,  1,  0, -2, -1,  0, -2,  1, -2,  0, -1,  0,
        -2,  0, -1,  1, -2, -2,  1,  0,  2], dtype=int8),
 array([ 3,  1, -1,  4, -1,  2, -8, -2, -1,  8], dtype=int8)]

In [4]:
conv_weights=[torch.int_repr(model_quantized.conv1.weight().cpu()).numpy(),torch.int_repr(model_quantized.conv2.weight().cpu()).numpy()]
conv_weights

[array([[[[  98,  106,  -30,  117,  -28],
          [  26,  -62,   75,  112,  -94],
          [ 111,   24,   94,   17,   62],
          [ -18,   98,   19,  -60,   33],
          [ -59,  -15,  -52,   85, -101]],
 
         [[   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0]],
 
         [[   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0]],
 
         [[   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0]]],
 
 
        [[[ -59,  -36,  -77,   12, -126],
          [ 115, -108,   98,   21,  -41],
          [  79,   20,  103,   14,  -40],
          [  34,  -

In [5]:
biases=[model_quantized.conv1.bias().detach().cpu().numpy(),model_quantized.conv2.bias().detach().cpu().numpy()]
conv_biases=[np.int8(biases[0]*100),np.int8(biases[1]*100)]
conv_biases

[array([ -4,   7, -17,  11,  -6, -15,   2,   9,  17, -12,  -9,   3,   5,
          7,  10,  -4,  11,  14,  -5,  14,   9,  17, -11,  -9,  -5,   1,
         12, -11,  10,  -7,  12,   0], dtype=int8),
 array([-4, -1, -5,  2, -2, -2,  0, -1,  4,  5, -5,  2,  0,  3, -2,  0,  0,
         2,  2,  2,  0, -4, -3, -3, -5,  5, -2, -3, -2,  3, -3, -4],
       dtype=int8)]

In [6]:
# non quantized cnn

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

torch.manual_seed(42)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(1152, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)


    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.softmax(self.fc2(x))

        return x

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Epoch 1, Loss: 1.6189
Epoch 2, Loss: 1.5767
Epoch 3, Loss: 1.5039
Epoch 4, Loss: 1.4766
Epoch 5, Loss: 1.4755
Test Accuracy: 98.61%


In [8]:
type(fcl_weights), type(fcl_biases)

(list, list)

In [10]:
len(fcl_weights), len(fcl_biases)

(2, 2)

In [11]:
len(fcl_weights[0]), len(fcl_weights[1]), len(fcl_biases[0]), len(fcl_biases[1])

(128, 10, 128, 10)

# Converting to COE

In [28]:
def make_coe(data, filename, radix = 2):
    """
    COE file with 8-bit signed binary
    """
    flat_data = data.flatten().tolist()

    with open(filename, 'w') as f:
        f.write(f"memory_initialization_radix={radix};\n")
        f.write("memory_initialization_vector=\n")

        for i, val in enumerate(flat_data):
            bin_str = format(val & 0xFF, '08b')

            if i < len(flat_data) - 1:
                f.write(bin_str + ",\n")
            else:
                f.write(bin_str + ";")

In [29]:
make_coe(fcl_weights[0], "fcl_weights_layer1.coe", radix = 2)
make_coe(fcl_weights[1], "fcl_weights_layer2.coe", radix = 2)

make_coe(fcl_biases[0], "fcl_biases_layer1.coe", radix = 2)
make_coe(fcl_biases[1], "fcl_biases_layer2.coe", radix = 2)

In [21]:
fcl_weights[0].flatten()

array([-95,  28,  -5, ...,  27, -60, -52], dtype=int8)

In [33]:
len(fcl_weights[0].flatten().tolist()) == 128*1152

True